In [0]:
import dlt
from pyspark import pipelines as dp
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
#data set rules 

dataset_rules = {
  "R1": "material_id is not null",
  "R2": "unit_cost > 0",
  "R3": "status is not null"
}

In [0]:
@dlt.table
@dlt.expect_all(dataset_rules)
def bronze_material_master():
    df = spark.readStream \
        .format("delta") \
        .option("readChangeFeed", "true") \
        .table("quiz.source.material_master")
    return df


In [0]:
@dlt.table
@dlt.expect_all_or_drop(dataset_rules)
def silver_mmd_data():
    df = spark.readStream.table("LIVE.bronze_material_master")
    df = df.withColumn("unit_cost", df.unit_cost.cast(DoubleType()))
    df = df.withColumn("last_updated", col("last_updated").cast(DateType()))
    df = df.withColumn("lead_time_days", col("lead_time_days").cast(IntegerType()))
    df = df.withColumn("safety_stock", col("safety_stock").cast(IntegerType()))
    df = df.withColumn("reorder_level", col("reorder_level").cast(IntegerType()))
    return df